In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
from ipywidgets import widgets
from sklearn import preprocessing
from tensorflow import keras 
from tensorflow.keras import layers, regularizers,Model, utils
%matplotlib inline 
import matplotlib.pyplot as plt
import sklearn
from tensorflow.keras.callbacks import TensorBoard
from datetime import time
import plotly.express as px
from sklearn.model_selection import train_test_split
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path
from livelossplot import PlotLossesKeras
import keras

In [ ]:
drivePaths = ["outputs/dataByLocation_2020-11-21-16-00-26_5FNYF6H05HB089022.csv"]
drivePaths = [str(path) for path in Path(".").rglob("outputs/dataByLocation*.csv")][:25]

In [ ]:
subsamplingPeriod = 1

drivesWithLocation = []
drivesWithoutLocation = []
for drivePath in drivePaths:
    drive = pd.read_csv(drivePath)
    drive = drive.iloc[::subsamplingPeriod]
    driveWithoutLocation = drive.drop(columns=["Time", "Longitude", "Latitude"])
    drivesWithLocation.append(drive)
    drivesWithoutLocation.append(driveWithoutLocation)

## Normalize Data

In [ ]:
normalizedDrives = []
for drive in drivesWithoutLocation:
    drive = drive.values[:]
    standard_scaler = preprocessing.StandardScaler()
    data_normalized = standard_scaler.fit_transform(drive)
    data_normalized = pd.DataFrame(data_normalized)
    normalizedDrives.append(data_normalized)

## Window Data

In [ ]:
sequenceLength = 50
features = ['Speed',
            'LatAcceleration',
            'LongAcceleration',
            'SteerTorque',
            'SteerRate',
            'SteerAngle',
            'FLWheelSpeed',
            'FRWheelSpeed',
            'RRWheelSpeed',
            'RLWheelSpeed']

windowedDrives = []
for drive in normalizedDrives:
    data_df = drive
    stackedData = []
    # split can_data into subsampled sequences
    for i in range(len(data_df)-sequenceLength):
        stackedData.append(data_df[i:i+sequenceLength])
    stackedData = np.array(stackedData)
    windowedDrives.append(stackedData)

## Generate Labels

In [ ]:
dataset = {"samples":[], "labels":[]}
for k,drive in enumerate(windowedDrives):
    for i,window in enumerate(drive[:-1]):
        last = drivesWithLocation[k].iloc[i]
        lastLong = last.Longitude
        lastLat = last.Latitude
        cur = drivesWithLocation[k].iloc[i+5]
        curLong = cur.Longitude
        curLat = cur.Latitude
        
        dataset["samples"].append(window)
        dataset["labels"].append([curLong - lastLong, curLat - lastLat])
dataset = pd.DataFrame(dataset)

### Normalize labels

In [ ]:
originalLabels = dataset["labels"].tolist()
# originalLabels = np.array(originalLabels)

scaler = preprocessing.MinMaxScaler()
labels_normalized = scaler.fit_transform(originalLabels)

dataset["labels"] = labels_normalized
labels_normalized.shape

### Formalize dataset

In [ ]:
samples = np.stack(dataset["samples"])
labels = labels_normalized
dataset = {"samples": samples, "labels": labels}

longitudeLabels = dataset["labels"][:,0]
latitudeLabels = dataset["labels"][:,1]

## Build Neural Network

### MLP

#### Split into training/validation

In [ ]:
longTrainInputs, longValInputs, longTrainLabels, longValLabels = train_test_split(dataset["samples"], longitudeLabels, test_size=0.5, shuffle=False)
latTrainInputs, latValInputs, latTrainLabels, latValLabels = train_test_split(dataset["samples"], latitudeLabels, test_size=0.5, shuffle=False)

#### Construct

In [ ]:
#specify input dimensionality
inputDimension = samples[0].shape[0]*samples[0].shape[1]
outputDimension = 1

#construct our neural network
hiddenLayerSizes = [32,128, 512, 512, 128]

#set up our input layer
inputLayer = layers.Input(shape=(sequenceLength, 13))

#set up our hidden layers
curLayer = 0
previousLayer = inputLayer
for i,curLayerSize in enumerate(hiddenLayerSizes):
    previousLayer = layers.Dense(curLayerSize,
                                 activation='linear', 
                                 name=str(curLayerSize)+'_hiddenLayer'+str(i)
                                )(previousLayer)
    previousLayer = layers.LeakyReLU(alpha=0.3)(previousLayer)
    previousLayer = layers.Dropout(
                                    rate=0.5
                                    )(previousLayer)
previousLayer = layers.Flatten()(previousLayer)
    
outputLayer = layers.Dense(outputDimension, activation='sigmoid')(previousLayer)

#### Compile

In [ ]:
#compile separate models
longModel = Model(inputs=inputLayer, outputs=[outputLayer], name='longitude_mlp')
latModel = Model(inputs=inputLayer, outputs=[outputLayer], name='latitude_mlp')
longModel.compile(loss='mean_squared_error', metrics=[tf.keras.metrics.RootMeanSquaredError()], optimizer='adam')
latModel.compile(loss='mean_squared_error', metrics=[tf.keras.metrics.RootMeanSquaredError()], optimizer='adam')

#### Train

In [ ]:
#train the mlp
longHistory = longModel.fit(longTrainInputs, longTrainLabels, epochs=3, batch_size=64, callbacks=[PlotLossesKeras()], validation_data=(longValInputs, longValLabels))
latHistory = latModel.fit(latTrainInputs, latTrainLabels, epochs=3, batch_size=64, callbacks=[PlotLossesKeras()], validation_data=(latValInputs, latValLabels))

#### Test

##### Predict Validation Set

In [ ]:
longPredictions = longModel.predict(longValInputs)
latPredictions = latModel.predict(latValInputs)

scatterData = pd.DataFrame({
    "index":np.arange(longPredictions.shape[0]),
    "long_predicted": longPredictions.reshape(longPredictions.shape[0]),
    "long_actual": longValLabels.reshape(longValLabels.shape[0]),
    "lat_predicted": latPredictions.reshape(latPredictions.shape[0]),
    "lat_actual": latValLabels.reshape(latValLabels.shape[0])
})

fig = go.Figure()
fig.add_trace(go.Scatter(x=scatterData.index, y=scatterData.long_predicted, name="long_predicted"))
fig.add_trace(go.Scatter(x=scatterData.index, y=scatterData.long_actual, name="long_actual"))
fig.add_trace(go.Scatter(x=scatterData.index, y=scatterData.lat_predicted, name="lat_predicted"))
fig.add_trace(go.Scatter(x=scatterData.index, y=scatterData.lat_actual, name="lat_actual"))

##### Predict Training Set

In [ ]:
longPredictions = longModel.predict(longTrainInputs)
latPredictions = latModel.predict(latTrainInputs)

scatterData = pd.DataFrame({
    "index":np.arange(longPredictions.shape[0]),
    "long_predicted": longPredictions.reshape(longPredictions.shape[0]),
    "long_actual": longTrainLabels.reshape(longTrainLabels.shape[0]),
    "lat_predicted": latPredictions.reshape(latPredictions.shape[0]),
    "lat_actual": latTrainLabels.reshape(latTrainLabels.shape[0])
})


fig = go.Figure()
fig.add_trace(go.Scatter(x=scatterData.index, y=scatterData.long_predicted, name="long_predicted"))
fig.add_trace(go.Scatter(x=scatterData.index, y=scatterData.long_actual, name="long_actual"))
fig.add_trace(go.Scatter(x=scatterData.index, y=scatterData.lat_predicted, name="lat_predicted"))
fig.add_trace(go.Scatter(x=scatterData.index, y=scatterData.lat_actual, name="lat_actual"))

### MLP (combined lat + long)

#### Split into training/validation

In [ ]:
combinedLabels = np.concatenate((np.reshape(longitudeLabels, (-1,1)), np.reshape(latitudeLabels, (-1,1))), axis=1)
trainInputs, valInputs, trainLabels, valLabels = train_test_split(dataset["samples"], combinedLabels, test_size=0.5, shuffle=False)

#### Construct

In [ ]:
#specify input dimensionality
inputDimension = samples[0].shape[0]*samples[0].shape[1]
outputDimension = 2

#construct our neural network
hiddenLayerSizes = [32]

#set up our input layer
inputLayer = layers.Input(shape=(sequenceLength, 13))

#set up our hidden layers
curLayer = 0
previousLayer = inputLayer
for i,curLayerSize in enumerate(hiddenLayerSizes):
    previousLayer = layers.Dense(curLayerSize,
                                 activation='linear', 
                                 name=str(curLayerSize)+'_hiddenLayer'+str(i)
                                )(previousLayer)
    previousLayer = layers.LeakyReLU(alpha=0.3)(previousLayer)
    previousLayer = layers.Dropout(
                                    rate=0.5
                                    )(previousLayer)
previousLayer = layers.Flatten()(previousLayer)
    
outputLayer = layers.Dense(outputDimension, activation='sigmoid')(previousLayer)

#### Compile

In [ ]:
#compile separate models
combinedModel = Model(inputs=inputLayer, outputs=[outputLayer], name='combined_mlp')
combinedModel.compile(loss='mean_squared_error', metrics=[tf.keras.metrics.RootMeanSquaredError()], optimizer='adam')

#### Train

In [ ]:
#train the mlp
combinedHistory = combinedModel.fit(trainInputs, trainLabels, epochs=25, batch_size=32, callbacks=[PlotLossesKeras()], validation_data=(valInputs, valLabels))

#### Test

##### Predict Validation Set

In [ ]:
predictions = combinedModel.predict(valInputs)

scatterData = pd.DataFrame({
    "index":np.arange(predictions[:,0].shape[0]),
    "long_predicted": predictions[:,0].reshape(predictions[:,0].shape[0]),
    "long_actual": valLabels[:,0].reshape(valLabels[:,0].shape[0]),
    "lat_predicted": predictions[:,1].reshape(predictions[:,1].shape[0]),
    "lat_actual": valLabels[:,1].reshape(valLabels[:,1].shape[0])
})

fig = go.Figure()
fig.add_trace(go.Scatter(x=scatterData.index, y=scatterData.long_predicted, name="long_predicted"))
fig.add_trace(go.Scatter(x=scatterData.index, y=scatterData.long_actual, name="long_actual"))
fig.add_trace(go.Scatter(x=scatterData.index, y=scatterData.lat_predicted, name="lat_predicted"))
fig.add_trace(go.Scatter(x=scatterData.index, y=scatterData.lat_actual, name="lat_actual"))

##### Predict Training Set

In [ ]:
predictions = combinedModel.predict(trainInputs)

scatterData = pd.DataFrame({
    "index":np.arange(predictions[:,0].shape[0]),
    "long_predicted": predictions[:,0].reshape(predictions[:,0].shape[0]),
    "long_actual": trainLabels[:,0].reshape(trainLabels[:,0].shape[0]),
    "lat_predicted": predictions[:,1].reshape(predictions[:,1].shape[0]),
    "lat_actual": trainLabels[:,1].reshape(trainLabels[:,1].shape[0])
})

fig = go.Figure()
fig.add_trace(go.Scatter(x=scatterData.index, y=scatterData.long_predicted, name="long_predicted"))
fig.add_trace(go.Scatter(x=scatterData.index, y=scatterData.long_actual, name="long_actual"))
fig.add_trace(go.Scatter(x=scatterData.index, y=scatterData.lat_predicted, name="lat_predicted"))
fig.add_trace(go.Scatter(x=scatterData.index, y=scatterData.lat_actual, name="lat_actual"))

### CNN

In [ ]:
trainingData.shape

In [ ]:
#specify input dimensionality
numberOfSamples = trainingData.shape[0]
inputDimension = trainingData[0].shape[0]*trainingData[0].shape[1]
outputDimension = 1

#construct our neural network
hiddenLayerSizes = [inputDimension, 32, 16, 8, 4, 2]

#set up our input layer
inputLayer = layers.Input(shape=(inputDimension, 1))

#set up our hidden layers
curLayer = 0
previousLayer = inputLayer
for curLayerSize in hiddenLayerSizes:
    previousLayer = layers.Conv1D(curLayerSize, 5,
                                  activation='sigmoid',
                                  name=str(curLayerSize)+'_hiddenLayer',
                                  kernel_regularizer=regularizers.L2(0.001),
                                  input_shape=(numberOfSamples, 5, 10)
                                 )(previousLayer)
    
outputLayer = layers.Dense(outputDimension, activation='sigmoid')(previousLayer)

#compile our model
ourModel = Model(inputs=inputLayer, outputs=[outputLayer], name='longitude_cnn')
ourModel.compile(loss='mean_squared_error', metrics=[tf.keras.metrics.RootMeanSquaredError()], optimizer='adam')
    

In [ ]:
#train the mlp
trainingDataFlat = trainingData.reshape((-1, inputDimension))
ourModel.fit(trainingDataFlat, longitudeLabels, epochs=10)

### CNN (new)

#### Split into training/validation

In [1]:
combinedLabels = np.concatenate((np.reshape(longitudeLabels, (-1,1)), np.reshape(latitudeLabels, (-1,1))), axis=1)
trainInputs, valInputs, trainLabels, valLabels = train_test_split(dataset["samples"], combinedLabels, test_size=0.5, shuffle=False)


NameError: name 'np' is not defined

#### Construct

In [ ]:
#specify input dimensionality
inputDimension = samples[0].shape[0]*samples[0].shape[1]
outputDimension = 2

#construct our neural network
hiddenLayerSizes = [inputDimension, 32, 32, 32, 32]

#set up our input layer
inputLayer = layers.Input(shape=(sequenceLength, 13))

#set up our hidden layers
curLayer = 0
previousLayer = inputLayer
for i,curLayerSize in enumerate(hiddenLayerSizes):
    previousLayer = layers.Conv1D(curLayerSize,
                                  kernel_size=3,
                                 activation='linear', 
                                 name=str(curLayerSize)+'_hiddenLayer'+str(i),
                                  kernel_regularizer=regularizers.L2(0.001)
                                )(previousLayer)
    previousLayer = layers.LeakyReLU(alpha=0.3)(previousLayer)
    previousLayer = layers.Dropout(
                                    rate=0.5
                                    )(previousLayer)
previousLayer = layers.Flatten()(previousLayer)
    
outputLayer = layers.Dense(outputDimension, activation='sigmoid')(previousLayer)

#### Compile

In [ ]:
#compile separate models
combinedModel = Model(inputs=inputLayer, outputs=[outputLayer], name='combined_mlp')
combinedModel.compile(loss='mean_squared_error', metrics=[tf.keras.metrics.RootMeanSquaredError()], optimizer='adam')

#### Train

In [ ]:
#train the mlp
combinedHistory = combinedModel.fit(trainInputs, trainLabels, epochs=5, batch_size=32, callbacks=[PlotLossesKeras()], validation_data=(valInputs, valLabels))

#### Test

##### Predict Validation Set

In [ ]:
predictions = combinedModel.predict(valInputs)

scatterData = pd.DataFrame({
    "index":np.arange(predictions[:,0].shape[0]),
    "long_predicted": predictions[:,0].reshape(predictions[:,0].shape[0]),
    "long_actual": valLabels[:,0].reshape(valLabels[:,0].shape[0]),
    "lat_predicted": predictions[:,1].reshape(predictions[:,1].shape[0]),
    "lat_actual": valLabels[:,1].reshape(valLabels[:,1].shape[0])
})

fig = go.Figure()
fig.add_trace(go.Scatter(x=scatterData.index, y=scatterData.long_predicted, name="long_predicted"))
fig.add_trace(go.Scatter(x=scatterData.index, y=scatterData.long_actual, name="long_actual"))
fig.add_trace(go.Scatter(x=scatterData.index, y=scatterData.lat_predicted, name="lat_predicted"))
fig.add_trace(go.Scatter(x=scatterData.index, y=scatterData.lat_actual, name="lat_actual"))

##### Predict Training Set

In [ ]:
predictions = combinedModel.predict(trainInputs)

scatterData = pd.DataFrame({
    "index":np.arange(predictions[:,0].shape[0]),
    "long_predicted": predictions[:,0].reshape(predictions[:,0].shape[0]),
    "long_actual": trainLabels[:,0].reshape(trainLabels[:,0].shape[0]),
    "lat_predicted": predictions[:,1].reshape(predictions[:,1].shape[0]),
    "lat_actual": trainLabels[:,1].reshape(trainLabels[:,1].shape[0])
})

fig = go.Figure()
fig.add_trace(go.Scatter(x=scatterData.index, y=scatterData.long_predicted, name="long_predicted"))
fig.add_trace(go.Scatter(x=scatterData.index, y=scatterData.long_actual, name="long_actual"))
fig.add_trace(go.Scatter(x=scatterData.index, y=scatterData.lat_predicted, name="lat_predicted"))
fig.add_trace(go.Scatter(x=scatterData.index, y=scatterData.lat_actual, name="lat_actual"))

### Resnet

In [ ]:
trainingData

In [ ]:
# sequences x samples x features

#specify input dimensionality
# numberOfSamples = trainingData.shape[0]
inputDimension = 50
outputDimension = 1

#construct our neural network
hiddenLayerSizes = [512]

#set up our input layer

#Will work for 2D convolution
inputLayer = layers.Input(shape=(5, 10, 1))
# inputLayer = layers.Input(shape=(5, 10))

#set up our hidden layers
curLayer = 0
previousPreviousLayer = None
previousLayer = inputLayer
for i,curLayerSize in enumerate(hiddenLayerSizes):
    previousPreviousLayer = previousLayer
    previousLayer = layers.Conv2D(curLayerSize, (3,3),
                                  activation='relu',
                                  name='hiddenLayer_'+str(i),
                                  kernel_regularizer=regularizers.L2(0.001),
                                  padding="same"
                                 )(tf.keras.layers.Concatenate(axis=1)([previousPreviousLayer, previousLayer]))
previousLayer = layers.Flatten()(previousLayer)
outputLayer = layers.Dense(outputDimension, activation='linear')(previousLayer)

#compile our model
opt = keras.optimizers.Adam(learning_rate=0.0001)
longModel = Model(inputs=inputLayer, outputs=[outputLayer], name='longitude_resnet')
latModel = Model(inputs=inputLayer, outputs=[outputLayer], name='latitude_resnet')
longModel.compile(loss='mean_absolute_error', metrics=[tf.keras.metrics.RootMeanSquaredError()], optimizer=opt)
latModel.compile(loss='mean_absolute_error', metrics=[tf.keras.metrics.RootMeanSquaredError()], optimizer=opt)
    

In [ ]:

# tensorboard = TensorBoard(log_dir='logs/{}'.format(time()))
print("Training longitude model:")
longModelHistory = longModel.fit(longTrainInputs, longTrainLabels, epochs=3, validation_data=(longValInputs, longValLabels))
print("Training latitude model:")
latModelHistory = latModel.fit(latTrainInputs, latTrainLabels, epochs=3, validation_data=(latValInputs, latValLabels))

#### Predictions

In [ ]:
longPredictions = longModel.predict(longValInputs)
latPredictions = latModel.predict(latValInputs)

In [ ]:
def deltasToAbsolute(deltas, starting=0):
    absolute = starting
    for delta in deltas:
        absolute+=delta
    return absolute

def deltasToAbsoluteSeries(deltas, starting=0):
    absolute = starting
    series = [starting]
    for delta in deltas:
        absolute+=delta
        series.append(absolute)
    return series

In [ ]:
scatterData = pd.DataFrame({
    "index":np.arange(longPredictions.shape[0]),
    "long_predicted": reverseNormalization(longPredictions.reshape(longPredictions.shape[0])),
    "long_actual": reverseNormalization(longValLabels.reshape(longValLabels.shape[0])),
    "lat_predicted": reverseNormalization(latPredictions.reshape(latPredictions.shape[0])),
    "lat_actual": reverseNormalization(latValLabels.reshape(latValLabels.shape[0]))
})
scatterData.describe()

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=scatterData.index, y=scatterData.long_predicted, name="long_predicted"))
fig.add_trace(go.Scatter(x=scatterData.index, y=scatterData.long_actual, name="long_actual"))
fig.add_trace(go.Scatter(x=scatterData.index, y=scatterData.lat_predicted, name="lat_predicted"))
fig.add_trace(go.Scatter(x=scatterData.index, y=scatterData.lat_actual, name="lat_actual"))

In [ ]:
latPredictions.all() == longPredictions.all()

### Combined long-lat resnet

In [ ]:
# sequences x samples x features

#specify input dimensionality
# numberOfSamples = trainingData.shape[0]
inputDimension = 50
outputDimension = 2

#construct our neural network
hiddenLayerSizes = [50, 32, 32, 16, 16, 8]

#set up our input layer

#Will work for 2D convolution
inputLayer = layers.Input(shape=(5, 10, 1))
# inputLayer = layers.Input(shape=(5, 10))

#set up our hidden layers
curLayer = 0
previousPreviousLayer = None
previousLayer = inputLayer
for i,curLayerSize in enumerate(hiddenLayerSizes):
    previousPreviousLayer = previousLayer
    previousLayer = layers.Conv2D(curLayerSize, (3,3),
                                  activation='relu',
                                  name='hiddenLayer_'+str(i),
                                  kernel_regularizer=regularizers.L2(0.001),
                                  padding="same"
                                 )(tf.keras.layers.Concatenate(axis=1)([previousPreviousLayer, previousLayer]))
previousLayer = layers.Flatten()(previousLayer)
outputLayer = layers.Dense(outputDimension, activation='linear')(previousLayer)

#compile our model
opt = keras.optimizers.Adam(learning_rate=0.0001)
combinedModel = Model(inputs=inputLayer, outputs=[outputLayer], name='combined_resnet')
combinedModel.compile(loss='mean_squared_error', metrics=[tf.keras.metrics.RootMeanSquaredError()], optimizer=opt)
    

In [ ]:
from sklearn.model_selection import train_test_split
longTrainInputs, longValInputs, longTrainLabels, longValLabels = train_test_split(trainingData, longitudeLabels, test_size=0.5, shuffle=False)
latTrainInputs, latValInputs, latTrainLabels, latValLabels = train_test_split(trainingData, latitudeLabels, test_size=0.5, shuffle=False)

combinedTrainLabels = np.array(tuple(zip(longTrainLabels, latTrainLabels)))
combinedValLabels = np.array(tuple(zip(longValLabels, latValLabels)))

# tensorboard = TensorBoard(log_dir='logs/{}'.format(time()))
print("Training combined model:")
combinedModelHistory = combinedModel.fit(longTrainInputs, combinedTrainLabels, epochs=10, validation_data=(longValInputs, combinedValLabels))

#### Predictions

In [ ]:
combinedPredictions = combinedModel.predict(longValInputs)

In [ ]:
combinedPredictions

In [ ]:
longPredictions = combinedPredictions[:,0]
latPredictions = combinedPredictions[:,1]

longPredictions

In [ ]:
latPredictions

In [ ]:
scatterData = pd.DataFrame({
    "index":np.arange(longPredictions.shape[0]),
    "long_predicted": longPredictions,
    "long_actual": longValLabels.reshape(longValLabels.shape[0]),
    "lat_predicted": latPredictions,
    "lat_actual": latValLabels.reshape(latValLabels.shape[0])
})
scatterData.describe()

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=scatterData.index, y=scatterData.long_predicted, name="long_predicted"))
fig1.add_trace(go.Scatter(x=scatterData.index, y=scatterData.long_actual, name="long_actual"))
fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=scatterData.index, y=scatterData.lat_predicted, name="lat_predicted"))
fig2.add_trace(go.Scatter(x=scatterData.index, y=scatterData.lat_actual, name="lat_actual"))

fig1

In [ ]:
fig2

In [ ]:
latPredictions.all() == longPredictions.all()

In [ ]:
latValLabels.all() == longValLabels.all()

### Combined Resnet with Absolute labels

In [ ]:
longitudeLabels = []
latitudeLabels = []

for i,window in enumerate(stackedData[:-1]):
    last = dataByLocation.iloc[i]
    lastLong = last.Longitude
    lastLat = last.Latitude
    cur = dataByLocation.iloc[i+5]
    curLong = cur.Longitude
    curLat = cur.Latitude
    
    longitudeLabels.append(curLong)
    latitudeLabels.append(curLat)

longitudeLabels = np.array(longitudeLabels)
latitudeLabels = np.array(latitudeLabels)

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=longitudeLabels, y=latitudeLabels))

In [ ]:
# sequences x samples x features

#specify input dimensionality
# numberOfSamples = trainingData.shape[0]
inputDimension = 50
outputDimension = 2

#construct our neural network
hiddenLayerSizes = [50, 32, 32, 16, 16, 8]

#set up our input layer

#Will work for 2D convolution
inputLayer = layers.Input(shape=(5, 10, 1))
# inputLayer = layers.Input(shape=(5, 10))

#set up our hidden layers
curLayer = 0
previousPreviousLayer = None
previousLayer = inputLayer
for i,curLayerSize in enumerate(hiddenLayerSizes):
    previousPreviousLayer = previousLayer
    previousLayer = layers.Conv2D(curLayerSize, (3,3),
#                                   activation='relu',
                                  name='hiddenLayer_'+str(i),
                                  kernel_regularizer=regularizers.L2(0.001),
                                  padding="same"
                                 )(tf.keras.layers.Concatenate(axis=1)([previousPreviousLayer, previousLayer]))
previousLayer = layers.Flatten()(previousLayer)
outputLayer = layers.Dense(outputDimension, activation='linear')(previousLayer)

#compile our model
opt = keras.optimizers.Adam(learning_rate=0.001)
combinedModel = Model(inputs=inputLayer, outputs=[outputLayer], name='absolute_combined_resnet')
combinedModel.compile(loss='mean_squared_error', metrics=[tf.keras.metrics.RootMeanSquaredError()], optimizer=opt)
    

In [ ]:
from sklearn.model_selection import train_test_split
longTrainInputs, longValInputs, longTrainLabels, longValLabels = train_test_split(trainingData, longitudeLabels, test_size=0.5, shuffle=False)
latTrainInputs, latValInputs, latTrainLabels, latValLabels = train_test_split(trainingData, latitudeLabels, test_size=0.5, shuffle=False)

combinedTrainLabels = np.array(tuple(zip(longTrainLabels, latTrainLabels)))
combinedValLabels = np.array(tuple(zip(longValLabels, latValLabels)))

# tensorboard = TensorBoard(log_dir='logs/{}'.format(time()))
print("Training combined model:")
combinedModelHistory = combinedModel.fit(longTrainInputs, combinedTrainLabels, epochs=10, validation_data=(longValInputs, combinedValLabels))

#### Predictions

In [ ]:
combinedPredictions = combinedModel.predict(longValInputs)

longPredictions = combinedPredictions[:,0]
latPredictions = combinedPredictions[:,1]

scatterData = pd.DataFrame({
    "index":np.arange(longPredictions.shape[0]),
    "long_predicted": longPredictions,
    "long_actual": longValLabels.reshape(longValLabels.shape[0]),
    "lat_predicted": latPredictions,
    "lat_actual": latValLabels.reshape(latValLabels.shape[0])
})
scatterData.describe()

import plotly.express as px
import plotly.graph_objects as go

fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=scatterData.index, y=scatterData.long_predicted, name="long_predicted"))
fig1.add_trace(go.Scatter(x=scatterData.index, y=scatterData.long_actual, name="long_actual"))
fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=scatterData.index, y=scatterData.lat_predicted, name="lat_predicted"))
fig2.add_trace(go.Scatter(x=scatterData.index, y=scatterData.lat_actual, name="lat_actual"))

fig1

In [ ]:
fig2